# Notebook 1: Data Sourcing & High-Fidelity Generation

## 1. Objective

The foundation of any analytics project is its data. A significant challenge in mining analytics is the lack of public, high-quality data, as real-world geotechnical sensor data is proprietary.

To overcome this, this notebook creates a **high-fidelity synthetic dataset**. We will follow a professional workflow:

1.  **Sourcing:** We will load and analyze two real-world Kaggle datasets (one for rainfall, one for seismic activity) to understand their statistical properties.
2.  **Generation:** We will use these real-world properties as our "recipe" to build a new, clean dataset of 20,000 samples.
3.  **Logic:** We will engineer purposeful, complex relationships between the features to create a rich dataset for analysis.

This notebook's final output is the `rockfall_synthetic_data.csv` file, which will be the foundation for the entire project.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Set a random seed for reproducibility
np.random.seed(42)

## 2. Part A: Sourcing & Analyzing "Driver" Datasets

To build a realistic synthetic dataset, we must first understand the properties of real-world "driver" factors. We will load and analyze two datasets from Kaggle to create our "recipe".

**Datasets Used:**
1.  **Rainfall:** "Rainfall Dataset for Simple Time Series Analysis"
2.  **Seismic:** "All the Earthquakes Dataset : from 1990-2023"

We will analyze their statistical distributions to ensure our synthetic data is not just random, but is statistically representative of real-world patterns.

In [6]:
import kaggle
import zipfile
import time

# ---
# ### 2.1. Setup File Paths & Download Data
# ---

# This is the path to our main project directory (one level up from 'notebooks')
BASE_DIR = '..'

# --- Define Data Directories ---
DATA_DIR = os.path.join(BASE_DIR, 'data')
os.makedirs(DATA_DIR, exist_ok=True) # Create 'data' folder if it doesn't exist
print(f"Data directory set to: {DATA_DIR}")


# --- Define Kaggle Dataset "Slugs" (from their URL) ---
RAINFALL_SLUG = 'sujithmandala/rainfall-dataset-for-simple-time-series-analysis'
SEISMIC_SLUG = 'alessandrolobello/the-ultimate-earthquake-dataset-from-1990-2023'

# --- Define our Standard File Names ---
RAINFALL_DRIVER_FILE = os.path.join(DATA_DIR, 'rainfall.csv')
SEISMIC_DRIVER_FILE = os.path.join(DATA_DIR, 'earthquake_data.csv')
SYNTHETIC_DATA_FILE = os.path.join(DATA_DIR, 'rockfall_synthetic_data.csv')

# --- Define the *actual* downloaded name for the seismic file ---
# We found this name from your "debug" cell output.
DOWNLOADED_SEISMIC_NAME = os.path.join(DATA_DIR, 'Eartquakes-1990-2023.csv')


# --- Download Logic ---
# This code will check if the file already exists. If not, it will download it.

# 1. Download Rainfall Data
if not os.path.exists(RAINFALL_DRIVER_FILE):
    print(f"Downloading {RAINFALL_SLUG}...")
    try:
        kaggle.api.dataset_download_files(RAINFALL_SLUG, path=DATA_DIR, unzip=True)
        print(f"Rainfall data downloaded and unzipped to {DATA_DIR}")
    except Exception as e:
        print(f"Error downloading rainfall data: {e}")
        print("Please check your Kaggle API setup (kaggle.json).")
else:
    print(f"Rainfall data ('{os.path.basename(RAINFALL_DRIVER_FILE)}') already exists. Skipping download.")

# 2. Download and Rename Seismic Data
if not os.path.exists(SEISMIC_DRIVER_FILE):
    print(f"Checking for seismic data...")
    
    # Check if the *original downloaded file* exists (e.g., Eartquakes-1990-2023.csv)
    if os.path.exists(DOWNLOADED_SEISMIC_NAME):
        print(f"Found '{os.path.basename(DOWNLOADED_SEISMIC_NAME)}'. Renaming...")
        os.rename(DOWNLOADED_SEISMIC_NAME, SEISMIC_DRIVER_FILE)
        print(f"Successfully renamed to '{os.path.basename(SEISMIC_DRIVER_FILE)}'.")
    
    # If neither file exists, then we need to download it
    else:
        print(f"Downloading {SEISMIC_SLUG}...")
        try:
            kaggle.api.dataset_download_files(SEISMIC_SLUG, path=DATA_DIR, unzip=True)
            print("Seismic data downloaded and unzipped.")
            
            # Give the system a second to make sure the file is written
            time.sleep(2) 
            
            # Now, do the rename
            if os.path.exists(DOWNLOADED_SEISMIC_NAME):
                os.rename(DOWNLOADED_SEISMIC_NAME, SEISMIC_DRIVER_FILE)
                print(f"Successfully renamed '{os.path.basename(DOWNLOADED_SEISMIC_NAME)}' to '{os.path.basename(SEISMIC_DRIVER_FILE)}'.")
            else:
                print(f"Warning: Downloaded seismic data, but the file '{os.path.basename(DOWNLOADED_SEISMIC_NAME)}' was not found for renaming.")
        
        except Exception as e:
            print(f"Error downloading seismic data: {e}")
            print("Please check your Kaggle API setup (kaggle.json).")
else:
    print(f"Seismic data ('{os.path.basename(SEISMIC_DRIVER_FILE)}') already exists. Skipping download.")
    
print("\n--- Data Sourcing and Setup Complete ---")
print(f"We are ready to analyze:")
print(f"1. {os.path.basename(RAINFALL_DRIVER_FILE)}")
print(f"2. {os.path.basename(SEISMIC_DRIVER_FILE)}")

Data directory set to: ..\data
Rainfall data ('rainfall.csv') already exists. Skipping download.
Seismic data ('earthquake_data.csv') already exists. Skipping download.

--- Data Sourcing and Setup Complete ---
We are ready to analyze:
1. rainfall.csv
2. earthquake_data.csv


### 2.2. "Recipe" for Rainfall

Now we will load the `rainfall.csv` file and analyze its statistical distribution. This will be the first part of our "recipe" for building our synthetic data. Our goal is to see what a "real-world" rainfall pattern looks like.

In [7]:
# ---
# ### 2.3. Load and Analyze Rainfall Data
# ---

try:
    # Load the CSV file into a pandas DataFrame
    # We defined RAINFALL_DRIVER_FILE in Cell 4
    df_rain = pd.read_csv(RAINFALL_DRIVER_FILE)

    print("--- Rainfall Data Analysis ---")
    
    # 1. Get Key Statistics
    # .describe() gives us a quick summary (count, mean, min, max, etc.)
    rainfall_stats = df_rain['rainfall'].describe()
    print("Statistical Summary of Daily Rainfall (mm):")
    print(rainfall_stats)
    print("\n")

    # 2. Calculate "Zero-Inflation"
    # This is a key part of our "recipe": What percentage of days have no rain?
    total_days = len(df_rain)
    no_rain_days = len(df_rain[df_rain['rainfall'] == 0])
    percent_no_rain = (no_rain_days / total_days) * 100
    
    # We will save this finding to use later in our generator
    RAINFALL_ZERO_PERCENT = percent_no_rain / 100  # Save as a decimal (e.g., 0.85)
    
    print(f"Total days analyzed: {total_days}")
    print(f"Days with 0mm rain: {no_rain_days}")
    print(f"Finding 1: {percent_no_rain:.2f}% of days have no rain (Zero-Inflated).")
    print("\n")
    
    # 3. Get "Extreme Event" Value
    # What does a "heavy" rain day look like? Let's check the 95th percentile.
    heavy_rain_threshold = df_rain['rainfall'].quantile(0.95)
    print(f"Finding 2: 95% of rain days are below {heavy_rain_threshold:.2f} mm.")


    # 4. Create a DAV Visualization
    # We use Plotly for an interactive histogram
    print("Generating interactive histogram...")
    fig = px.histogram(df_rain, 
                       x='rainfall', 
                       nbins=100, 
                       title='Distribution of Real-World Daily Rainfall (mm)',
                       labels={'rainfall': 'Rainfall in mm'})
    
    # Improve the layout
    fig.update_layout(yaxis_title='Number of Days',
                      xaxis_title='Rainfall in mm (log scale)',
                      xaxis_type="log") # Use a log scale to see the tail better
    
    fig.show()

except FileNotFoundError:
    print(f"Error: The file '{RAINFALL_DRIVER_FILE}' was not found.")
    print("Please re-run Cell 4 to download the data.")
except Exception as e:
    print(f"An error occurred: {e}")

--- Rainfall Data Analysis ---
Statistical Summary of Daily Rainfall (mm):
count    119.000000
mean       9.484034
std        6.860992
min        0.000000
25%        4.050000
50%        9.500000
75%       14.850000
max       22.400000
Name: rainfall, dtype: float64


Total days analyzed: 119
Days with 0mm rain: 24
Finding 1: 20.17% of days have no rain (Zero-Inflated).


Finding 2: 95% of rain days are below 21.16 mm.
Generating interactive histogram...
